## SARSA example of Frozen Lake

In [1]:
USE {
    repositories {
        mavenCentral()
        maven("https://central.sonatype.com/repository/maven-snapshots/")
    }
    dependencies {
        implementation("io.github.kotlinrl:integration:0.1.0-SNAPSHOT")
        implementation("io.github.kotlinrl:tabular:0.1.0-SNAPSHOT")
        implementation("io.github.kotlinrl:envs:0.1.0-SNAPSHOT")
        implementation("io.github.kotlinrl:rendering:0.1.0-SNAPSHOT")
    }
}

In [2]:
import io.github.kotlinrl.core.*
import io.github.kotlinrl.integration.gymnasium.*
import io.github.kotlinrl.integration.gymnasium.GymnasiumEnvs.*
import io.github.kotlinrl.rendering.*
import io.github.kotlinrl.tabular.*
import org.jetbrains.kotlinx.kandy.letsplot.export.*
import org.jetbrains.kotlinx.multik.api.*
import org.jetbrains.kotlinx.multik.api.io.*
import org.jetbrains.kotlinx.multik.ndarray.data.*
import java.io.*


In [3]:
val maxStepsPerEpisode = 200
val trainingEpisodes = 20_000
val testEpisodes = 50
val initialEpsilon = 0.9
val minEpsilon = 0.0
val epsilonDecayRate = (initialEpsilon - minEpsilon) / (trainingEpisodes * 0.8)
val alpha = 0.2
val gamma = 0.99
val fileName = "FrozenLakeQLearning.npy"
val actionSymbols = mapOf(
    0 to "←",
    1 to "↓",
    2 to "→",
    3 to "↑"
)


In [4]:
val env = gymnasium.make<FrozenLakeEnv>(
    FrozenLake_v1, render = true, options = mapOf(
        "is_slippery" to false,
        "map_name" to "8x8"
    )
)

val trainingQtable: QTable = mk.rand<Double, D2>(from = 0.24, until = 0.26, dims = intArrayOf(64, 4))

val (epsilonSchedule, epsilonDecrement) = ParameterSchedule.linearDecay(
    initialValue = initialEpsilon,
    minValue = minEpsilon,
    decayRate = epsilonDecayRate,
    callback = { episode, parameter ->
        if (episode % 1000 == 0) {
            println("Episode: $episode, Epsilon: $parameter")
        }
    }
)


2025-09-19T22:51:29.488137Z Execution of code 'val env = gymnasium....' ERROR Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


In [5]:
val trainer = episodicTrainer(
    env = env,
    agent = learningAgent(
        id = "training",
        algorithm = QLearning(
            Q = trainingQtable,
            epsilon = epsilonSchedule,
            alpha = ParameterSchedule.constant(alpha),
            gamma = gamma,
        )
    ),
    maxStepsPerEpisode = maxStepsPerEpisode,
    successfulTermination = { it.reward == 1.0 },
    callbacks = listOf(
        printEpisodeStart(1000),
        onEpisodeEnd {
            epsilonDecrement()
            if (it.totalEpisodes % 1_000 == 0) {
                val goalSuccessCount = TrainingResult(it.episodeStats.takeLast(1_000)).totalGoalSuccessCount
                println("Current goal success count: $goalSuccessCount, over the last 1000 episodes")
            }
        }
    )
)
println("Starting training")
val training = trainer.train(maxEpisodes(trainingEpisodes).or {
    it.totalEpisodes >= 1000 && it.takeLast(1000).totalGoalSuccessCount == 1000
})
mk.writeNPY(fileName, trainingQtable)


Starting training
Starting episode 1000
Episode: 1000, Epsilon: Parameter(current=0.8437500000000201, previous=0.8438062500000201, minValue=0.0, decayStep=1000)
Current goal success count: 9, over the last 1000 episodes
Starting episode 2000
Episode: 2000, Epsilon: Parameter(current=0.7875000000000402, previous=0.7875562500000401, minValue=0.0, decayStep=2000)
Current goal success count: 62, over the last 1000 episodes
Starting episode 3000
Episode: 3000, Epsilon: Parameter(current=0.7312500000000602, previous=0.7313062500000602, minValue=0.0, decayStep=3000)
Current goal success count: 68, over the last 1000 episodes
Starting episode 4000
Episode: 4000, Epsilon: Parameter(current=0.6750000000000803, previous=0.6750562500000803, minValue=0.0, decayStep=4000)
Current goal success count: 80, over the last 1000 episodes
Starting episode 5000
Episode: 5000, Epsilon: Parameter(current=0.6187500000001004, previous=0.6188062500001004, minValue=0.0, decayStep=5000)
Current goal success count: 

In [6]:
val testingQtable = mk.readNPY<Double, D2>(fileName).asD2Array()


In [7]:
val recordEnv = RecordVideo(env = env, folder = "videos/frozen_lake_q_learning", testEpisodes / 3)
val tester = episodicTrainer(
    env = recordEnv,
    agent = policyAgent(
        id = "testing",
        policy = testingQtable.greedy()
    ),
    maxStepsPerEpisode = maxStepsPerEpisode,
    successfulTermination = { it.done },
    callbacks = listOf(
        printEpisodeStart(10)
    )
)
println("Starting testing")
val test = tester.train(maxEpisodes(testEpisodes))


Starting testing
Starting episode 10
Starting episode 20
Starting episode 30
Starting episode 40
Starting episode 50
Max episodes reached: 50


In [8]:
println("Training average reward: ${training.totalAverageReward}")
println("Test average reward: ${test.totalAverageReward}")

printQTable(testingQtable, 8, 8, actionSymbols = actionSymbols)
displayVideos(recordEnv.folder)


Training average reward: 0.3606605721026246
Test average reward: 1.0
Action Value Function:
  0.88   0.89   0.90   0.90   0.91   0.92   0.93   0.94 
  0.89   0.90   0.90   0.91   0.92   0.93   0.94   0.95 
  0.90   0.90   0.91   0.26   0.93   0.94   0.95   0.96 
  0.90   0.91   0.92   0.93   0.94   0.26   0.96   0.97 
  0.90   0.90   0.91   0.25   0.95   0.96   0.97   0.98 
  0.89   0.26   0.25   0.93   0.94   0.95   0.26   0.99 
  0.88   0.26   0.25   0.69   0.26   0.94   0.26   1.00 
  0.71   0.25   0.25   0.26   0.32   0.80   0.26   0.26 
Policy Table:
  ↓   ↓   ↓   ↓   ↓   ↓   ↓   ↓ 
  ↓   ↓   ↓   →   ↓   ↓   ↓   ↓ 
  ↓   ↓   ↓   ↑   ↓   →   ↓   ↓ 
  →   →   →   →   ↓   ←   ↓   ↓ 
  →   →   ↑   →   →   →   →   ↓ 
  ↑   →   ↑   →   ↑   ↑   ↓   ↓ 
  ↑   ←   →   ↑   ↓   ↑   ↓   ↓ 
  ↑   →   ↑   →   →   ↑   ↓   ← 


In [9]:
plotPolicyActionValueGrid(testingQtable, 8, 8, actionSymbols )

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gCfnBi"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"Action Value Function"
},
"mapping":{
},
"data":{
"x":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0],
"y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-6.0,-6.0,-6.0,-6.0,-6.0,-6.0,-6.0,-6.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0],
"label":["0.88","0.89","0.9","0.9","0.91","0.92","0.93","0.94","0.89","0.9","0.9","0.91","0.92","0.93","0.94","0.95","0.9","0.9","0.91","0.26","0.93","0.94","0.95","0.96","0.9","0.91","0.92","0.93","0.94","0.26","0.96","0.97","0.9","0.9","0.91","0.25","0.95","0.96","0.97","0.98","0.89","0.26","0.25","0.93","0.94","0.95","0.26","0.99","0.88","0.26","0.25","0.69","0.26","0.94","0.26","1.0","0.71","0.25","0.25","0.26","0.32","0.8","0.26","0.26"],
"value":[0.88,0.89,0.9,0.9,0.91,0.92,0.93,0.94,0.89,0.9,0.9,0.91,0.92,0.93,0.94,0.95,0.9,0.9,0.91,0.26,0.93,0.94,0.95,0.96,0.9,0.91,0.92,0.93,0.94,0.26,0.96,0.97,0.9,0.9,0.91,0.25,0.95,0.96,0.97,0.98,0.89,0.26,0.25,0.93,0.94,0.95,0.26,0.99,0.88,0.26,0.25,0.69,0.26,0.94,0.26,1.0,0.71,0.25,0.25,0.26,0.32,0.8,0.26,0.26]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ffffff",
"low":"#5470c6",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"fill":"value"
},
"stat":"identity",
"size":0.5,
"color":"#000000",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"tile",
"data":{
}
},{
"mapping":{
"x":"x",
"y":"y",
"label":"label"
},
"stat":"identity",
"size":6.0,
"color":"#000000",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"text",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"x"
},{
"type":"int",
"column":"y"
},{
"type":"float",
"column":"value"
},{
"type":"str",
"column":"label"
}]
},
"spec_id":"3"
},{
"ggtitle":{
"text":"Policy"
},
"mapping":{
},
"data":{
"x":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0],
"y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-6.0,-6.0,-6.0,-6.0,-6.0,-6.0,-6.0,-6.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0],
"label":["↓","↓","↓","↓","↓","↓","↓","↓","↓","↓","↓","→","↓","↓","↓","↓","↓","↓","↓","↑","↓","→","↓","↓","→","→","→","→","↓","←","↓","↓","→","→","↑","→","→","→","→","↓","↑","→","↑","→","↑","↑","↓","↓","↑","←","→","↑","↓","↑","↓","↓","↑","→","↑","→","→","↑","↓","←"],
"value":[0.88,0.89,0.9,0.9,0.91,0.92,0.93,0.94,0.89,0.9,0.9,0.91,0.92,0.93,0.94,0.95,0.9,0.9,0.91,0.26,0.93,0.94,0.95,0.96,0.9,0.91,0.92,0.93,0.94,0.26,0.96,0.97,